# Testing with Gensim, can skip to next section for final model

In [5]:
import gensim
from gensim.models import LdaMulticore, CoherenceModel, LdaModel, TfidfModel
from gensim.corpora import Dictionary

from joblib import load


In [ ]:
vec=load('Data/1gram-vectorizer.joblib')
vecs = vec.transform(data.tokes)

In [93]:
data.tokes[0:2]

0    [@ronnie_haydon, almost, certainly, mange, ., ...
1    [[, ivermectin, paste, dewormer, -, 6.08, g, d...
Name: tokes, dtype: object

In [12]:
dct = Dictionary(data.no_punc)  # fit dictionary data.tokes
corpus = [dct.doc2bow(line) for line in data.no_punc] 

In [13]:
for doc in corpus[0:2]:
   print([[dct[id], freq] for id, freq in doc])

[['@ronnie_haydon', 1], ['almost', 1], ['and', 1], ['are', 1], ['at', 1], ['certainly', 1], ['contact', 1], ['do', 1], ['eat', 1], ['for', 2], ['free', 1], ['give', 1], ['happy', 1], ['her', 2], ['i', 2], ['if', 1], ['in', 1], ['it', 2], ['ivermectin', 1], ['last', 1], ['leave', 1], ['local', 1], ['mange', 1], ['most', 1], ['put', 1], ['roughly', 1], ['same', 1], ['saw', 1], ['see', 1], ['some', 1], ['something', 1], ['tasty', 1], ['the', 1], ['then', 1], ['they', 1], ['this', 1], ['time', 1], ['to', 4], ['treat', 1], ['vet', 1], ['vets', 1], ['where', 1], ['will', 1], ['would', 2], ['you', 2], ['your', 1]]
[['ivermectin', 2], ['%', 2], ['-', 4], ['187', 2], ['3', 2], ['608', 2], ['@', 2], ['[', 1], [']', 1], ['apple', 2], ['been', 1], ['dewormer', 2], ['dose', 2], ['flavor', 2], ['g', 2], ['has', 1], ['https//tco/3aj1qfs4hn', 1], ['https//tco/rrloi8hw3n', 1], ['on', 1], ['pack', 2], ['paste', 2], ['pet', 1], ['posted', 1], ['supplies', 1], ['–', 1]]


In [20]:
data[data.no_punc == '']

,datetime,lang,rts,likes,quotes,replies,hr,date,tokes,no_punc


In [36]:
dct.filter_extremes(no_below=10, no_above=0.8, keep_n=100000)
new_corpus = [dct.doc2bow(doc) for doc in data.no_punc]

In [37]:
model = TfidfModel(new_corpus)

In [38]:
tf_corp = model[new_corpus]

In [40]:
lda = LdaMulticore(tf_corp, id2word=dct, num_topics=30, workers=9, iterations=6, eta='auto')
lda.save('1gram-t30-lda.model')

In [ ]:
lda.print_topics()

## Grid Searching Topics

In [1]:
import optuna
import gensim
import pandas as pd
from gensim.models import LdaMulticore, CoherenceModel, LdaModel, TfidfModel
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
import numpy as np
from joblib import load, dump

In [2]:
data=pd.read_csv('Data/no_punc_tokes.csv', index_col='Unnamed: 0')
data['no_punc'] = data['no_punc'].apply(lambda x: eval(x))
data=data[data['no_punc'].notnull()].reset_index().drop('index', axis=1).drop(['tokes', 'datetime'], axis=1)
data.head()

/tmp/ipykernel_403/1253446902.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('/mnt/c/Users/LiuD/OneDrive - University of Arkansas for Medical Sciences/Grad School/FA2022 - Project Rotation/Code/Data/no_punc_tokes.csv', index_col='Unnamed: 0')


,lang,rts,likes,quotes,replies,hr,date,no_punc
0,en,0.0,1.0,0.0,1,20.0,2020-03-01,"[@ronnie_haydon, almost, certainly, mange, i, ..."
1,en,0.0,0.0,0.0,0,17.0,2020-03-01,"[[, ivermectin, paste, dewormer, -, 608, g, do..."
2,en,0.0,8.0,0.0,0,12.0,2020-03-01,"[🏁, the, bohemia, team, is, gearing, up, to, s..."
3,en,1.0,4.0,0.0,0,15.0,2020-03-02,"[@ginagh, works, as, #isglobal, director, of, ..."
4,en,0.0,0.0,0.0,1,21.0,2020-03-03,"[@liezjkc, categorically, untrue, warren, is, ..."


In [3]:
bad = ["'", 't', 'you', 'i', 'it', 's', 'don', 'they', 'that', 'to', 'the', 'he', 'she', 'and', 'a', 'doesn', 'didn', 'if', 'but', '...', 'is', 'what', 'people', 'just', 'we', 'do', 'him', 'her', 'his', 'hers', 'of', 'is', '', ' ', 'people', 'are', 'for', 'a', '&', 'by', 'would', 'been', 'were', 'if', 'for', '"', '-', 'in', 'to', 'on', 'as', 'was', 'my', 'had', 'can', 'do', 'so', 'are', '%', '/', 'by', 'in', 'it', 'to', '|', "'", 'via', 'says', '*', 'of', ' ', 'that', 'from' "she's", '[', ']','i', 'a', 'you', 'he', 'it', 'they', 'take', 'is', 'for', 'of', 'on', 'but', 'this', 'my', 'your', 'yours', 'mine', 'me', 'be', 'like', 'have', 'some', 'had', 'with', 'here', 'them', 'there', 'without', 'be', 'm', 'has', "’"]
good = ['who','cdc', 'fda','ivermectin', 'horse', 'covid']

In [4]:
class Objective:

    def __init__(self, nts, nte):
        self.nts = nts
        self.nte = nte

    def __call__(self, trial):
        dct_no_below = trial.suggest_int('no_below', 800, 1200, step=50)
        dct_no_above = trial.suggest_float('no_above', 0.6, 0.8, step=0.05)
        dct_keep_n = trial.suggest_int('keep_n',50000, 100000, step=5000)
        #dct_bad = trial.suggest_categorical('bad_ids', [None, bad])
        #dct_good = trial.suggest_categorical('good_ids', [None, good])
        lda_num_topics = trial.suggest_int('num_topics', self.nts, self.nte, step=1)
        #lda_iterations = trial.suggest_int('iterations', 0, 10)
        #lda_eta = trial.suggest_categorical('eta',['symmetrical', 'auto'])
        #lda_passes = trial.suggest_int('passes', 0, 10, step=2)
        #lda_alpha = trial.suggest_categorical('alpha',['symmetrical', 'asymmetrical'])
        #lda_decay = trial.suggest_categorical('decay', [None, 0.5, 0.6, 0.7, 0.8, 0.9, 1])
        lda_minimum_probability = trial.suggest_float('minimum_probability', 0.2, 0.5, step=0.05)
        lda_per_word_topics = trial.suggest_categorical('per_word_topics',[True,False])

        dct = Dictionary(data.no_punc)  # fit dictionary data.tokes
        corpus = [dct.doc2bow(line) for line in data.no_punc]

        gt=[]
        for k in good:
            try:
                gt.append(dct.token2id[k])
            except:
                continue

        dct.filter_extremes(
            no_below=dct_no_below, 
            no_above=dct_no_above, 
            keep_n=dct_keep_n,
            keep_tokens=gt)

        bt=[]
        for j in list(set(bad)):
            try:
                bt.append(dct.token2id[j])
            except:
                continue

        dct.filter_tokens(
            bad_ids=bt
        )
        corpus = [dct.doc2bow(doc) for doc in data.no_punc]
        model = TfidfModel(corpus)
        lda = LdaMulticore(model[corpus], id2word=dct, workers=9, 
            num_topics=lda_num_topics, 
            #iterations=lda_iterations, 
            #alpha = 'symmetrical',  #ValueErrors when alpha is asymmetrical, or symmetrical, can't find alpha
            #decay = lda_decay,
            random_state=19,
            minimum_probability= lda_minimum_probability,
            per_word_topics = lda_per_word_topics,
            #eta=lda_eta
            )

        cm = CoherenceModel(model=lda, texts=data.no_punc, dictionary = dct, coherence='c_v')
        coherence = cm.get_coherence()

        return coherence

In [5]:
study = optuna.create_study(direction = 'maximize')

[I 2023-05-10 17:40:50,163] A new study created in memory with name: no-name-c40c9126-1f4c-49e6-9f44-9a6bbdad6ae1


In [ ]:
study.optimize(Objective(nts=13,nte=17), n_trials=10, gc_after_trial=True) #show_progress_bar=True, 

In [15]:
import joblib
joblib.dump(study, 'study.pkl')

['study.pkl']

In [16]:
study.best_params  # E.g. {'x': 2.002108042}

{'no_below': 1000,
 'no_above': 0.7,
 'keep_n': 95000,
 'num_topics': 15,
 'minimum_probability': 0.45,
 'per_word_topics': True}

In [17]:
dct = Dictionary(data.no_punc)  # fit dictionary data.tokes
corpus = [dct.doc2bow(line) for line in data.no_punc] 


gt=[]
for k in good:
    try:
        gt.append(dct.token2id[k])
    except:
        continue

dct.filter_extremes(
    no_below=study.best_params['no_below'], 
    no_above=study.best_params['no_above'], 
    keep_n=study.best_params['keep_n'],
    keep_tokens=gt)

bt=[]
for j in list(set(bad)):
    try:
        bt.append(dct.token2id[j])
    except:
        continue

dct.filter_tokens(
    bad_ids=bt
    )
corpus = [dct.doc2bow(doc) for doc in data.no_punc]
model = TfidfModel(corpus)
lda = LdaMulticore(model[corpus], id2word=dct, workers=9, 
    num_topics=study.best_params['num_topics'], 
    #iterations=0, 
    #alpha = lda_alpha,  #ValueErrors when alpha is asymmetrical, or symmetrical, can't find alpha
    #decay = lda_decay,
    minimum_probability= study.best_params['minimum_probability'],
    random_state=19,
    per_word_topics = study.best_params['per_word_topics']
    #eta=lda_eta
    )
cm = CoherenceModel(model=lda, texts=data.no_punc, dictionary = dct, coherence='c_v')
coherence = cm.get_coherence()

In [18]:
coherence

0.4717672728691147

In [19]:
from joblib import dump
dump(corpus, 'Data/T15 Gensim/corpus.joblib')
dump(model, 'Data/T15 Gensim/tfidf_model.joblib')
dump(lda, 'Data/T15 Gensim/15t_lda_model.joblib')

['/mnt/c/Users/LiuD/OneDrive - University of Arkansas for Medical Sciences/Grad School/FA2022 - Project Rotation/Code/Data/T15 Gensim/15t_lda_model.joblib']

In [22]:
dump(dct, 'Data/T15 Gensim/dct.joblib')

['/mnt/c/Users/LiuD/OneDrive - University of Arkansas for Medical Sciences/Grad School/FA2022 - Project Rotation/Code/Data/T15 Gensim/dct.joblib']

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
pyLDAvis.enable_notebook()

vis_data = gensimvis.prepare(lda, model[corpus], dct)
pyLDAvis.save_html(vis_data, 'Data/T15 Gensim/ldavis.html')
pyLDAvis.display(vis_data)

In [ ]:
vis_data = gensimvis.prepare(lda, model[corpus], dct, mds='tsne')
pyLDAvis.save_html(vis_data, 'Data/T15 Gensim/1gram-ldavis-t15-tsne.html')
pyLDAvis.display(vis_data)

/root/miniconda3/envs/nlp/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/root/miniconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/nlp/lib/python3.9/site-

In [ ]:
vis_data = gensimvis.prepare(lda, model[corpus], dct, mds='mmds')
pyLDAvis.save_html(vis_data, 'Data/T15 Gensim/1gram-ldavis-t15-mmds.html')
pyLDAvis.display(vis_data)

/root/miniconda3/envs/nlp/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/root/miniconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/nlp/lib/python3.9/site-

In [1]:
lda.print_topics()

NameError: name 'lda' is not defined

In [ ]:
lda.save('Data/T15 Gensim/lda_model.joblib')

In [ ]:
lda.top_topics(corpus)

[([(0.010958718, '$'),
   (0.010607295, 'pharma'),
   (0.010581819, 'big'),
   (0.008268868, 'money'),
   (0.007980562, '’'),
   (0.007379754, 'pfizer'),
   (0.006256068, 'make'),
   (0.0059907273, 'merck'),
   (0.005351514, 'covid'),
   (0.0052908733, 'new'),
   (0.0052506123, 'no'),
   (0.0051986217, 'not'),
   (0.005183462, 'cheap'),
   (0.004993583, 'pill'),
   (0.004882597, 'drug'),
   (0.004374229, 'why'),
   (0.0043441793, 'made'),
   (0.00423412, 'effective'),
   (0.00409076, 'patent'),
   (0.0040877326, 'from')],
  -2.6625342915465713),
 ([(0.008500774, '’'),
   (0.0070435735, 'not'),
   (0.00678406, '@pierrekory'),
   (0.006490553, 'side'),
   (0.006080848, 'effects'),
   (0.0055087856, 'no'),
   (0.005442566, 'covid'),
   (0.0047924086, 'than'),
   (0.004685946, 'work'),
   (0.004317628, 'or'),
   (0.0040277396, 'more'),
   (0.0040127, 'about'),
   (0.0038088895, 'studies'),
   (0.0035564005, 'long'),
   (0.0035145748, 'at'),
   (0.003504876, 'from'),
   (0.0035000453, 'all'

In [18]:
data.head()

,lang,rts,likes,quotes,replies,hr,date,no_punc
0,en,0.0,1.0,0.0,1,20.0,2020-03-01,"[@ronnie_haydon, almost, certainly, mange, i, ..."
1,en,0.0,0.0,0.0,0,17.0,2020-03-01,"[[, ivermectin, paste, dewormer, -, 608, g, do..."
2,en,0.0,8.0,0.0,0,12.0,2020-03-01,"[🏁, the, bohemia, team, is, gearing, up, to, s..."
3,en,1.0,4.0,0.0,0,15.0,2020-03-02,"[@ginagh, works, as, #isglobal, director, of, ..."
4,en,0.0,0.0,0.0,1,21.0,2020-03-03,"[@liezjkc, categorically, untrue, warren, is, ..."


In [ ]:
def topic_pred(tokes):
    new_doc_bow = dct.doc2bow(tokes)
    topics = lda.get_document_topics(new_doc_bow)
    if topics == []:
        return 99
    else:
        return topics[0][0]

In [ ]:
data['topics']=data.no_punc.apply(topic_pred)

In [70]:
data.head()

,lang,rts,likes,quotes,replies,hr,date,no_punc,topics
0,en,0.0,1.0,0.0,1,20.0,2020-03-01,"[@ronnie_haydon, almost, certainly, mange, i, ...",7.0
1,en,0.0,0.0,0.0,0,17.0,2020-03-01,"[[, ivermectin, paste, dewormer, -, 608, g, do...",7.0
2,en,0.0,8.0,0.0,0,12.0,2020-03-01,"[🏁, the, bohemia, team, is, gearing, up, to, s...",1.0
3,en,1.0,4.0,0.0,0,15.0,2020-03-02,"[@ginagh, works, as, #isglobal, director, of, ...",11.0
4,en,0.0,0.0,0.0,1,21.0,2020-03-03,"[@liezjkc, categorically, untrue, warren, is, ...",3.0


In [ ]:
topics_dates=data[['topics', 'date']]

In [ ]:
topics_dates.to_csv('Data/T15 Gensim/topics_dates.csv')

In [80]:
import pandas as pd
topics_dates = pd.read_csv('Data/13T Gensim/topics_dates.csv')

In [83]:
top_plot=topics_dates.groupby(['date', 'topics']).count()
top_plot.index.get_level_values('date')
import plotly.express as px
fig = px.line(topics_dates.groupby(['date', 'topics']).count(), x = top_plot.index.get_level_values('date'), y='Unnamed: 0', color=top_plot.index.get_level_values('topics'), width=1500, height=600)
fig.show()

/root/miniconda3/envs/nlp/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/root/miniconda3/envs/nlp/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

